### **Install and Imports**

In [ ]:
# !pip install numpy emoji langdetect contractions beautifulsoup4 pandas nltk


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup # BeautifulSoup for parsing HTML and XML documents (Richardson, 2025)
import re # Regular expression library for text processing
import emoji
import textwrap # textwrap library for formatting text
from langdetect import detect, DetectorFactory
from collections import Counter
import pandas as pd
# NLP processing libraries # (NLTK Project, 2024)
# Natural Language Toolkit (NLTK) is a suite of libraries and programs for
# statistical natural language processing (NLP) 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
import contractions 
from langdetect import detect, detect_langs, DetectorFactory
from collections import Counter
from emoji import EMOJI_DATA
from concurrent.futures import ProcessPoolExecutor

### **Read from CSV file: 2020**


In [5]:
# show full column content
pd.set_option("display.max_colwidth", None)  

#### **Data Merged From**

Data Merged From 
- hashtag_donaldtrump.csv
- hashtag_joebiden.csv

In [ ]:
# common_args = dict(engine="python", encoding="utf-8", encoding_errors="replace")
# df1 = pd.read_csv("Raw Data/hashtag_donaldtrump.csv", **common_args)
# df2 = pd.read_csv("Raw Data/hashtag_joebiden.csv", **common_args)

In [ ]:
# columns_to_drop = [col for col in df1.columns if col not in ['tweet']]
# df1 = df1.drop(columns=columns_to_drop)
# columns_to_drop = [col for col in df2.columns if col not in ['tweet']]
# df2 = df2.drop(columns=columns_to_drop)


In [ ]:
# df= pd.concat([df1, df2], ignore_index=True)
# df = df.sample(frac=1, random_state=42).reset_index(drop=True)
# df.to_csv("Raw Data/2020_Sample_Complete.csv", index=False) # full data for later use
# # Confirm structure
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225747 entries, 0 to 1225746
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   tweet   1225683 non-null  object
dtypes: object(1)
memory usage: 9.4+ MB


#### **Data Merged From**

In [116]:
df = pd.read_csv("Raw Data/2020_Sample_Complete.csv")
df.head()

,tweet
0,Tricky #Biden. #Elections2020 https://t.co/syPe9u7JgF
1,#HumanRights #StopArmenianAggression #Paris #StopArmenianTerrorism #Trump #Germania https://t.co/XgSKkDeiPi
2,The Conservative Movement has become a dangerous cult. Republicans are NOT going to listen to health experts??? Wow. It will be a long time getting over this pandemic. #Trump #COVIDIOTS #Cult45 #TrumpIsCompromised #VoteThemAllOut #BlueWave https://t.co/7uSF58DRta
3,"As it turns out, #Trump was right as usual. He said the problem w/tearing down statues is... where does it end? Who'll be next George Washington? Three years later, that's exactly what's happened. Lunatics want to tear down even Lincoln's statue. Washington was mentioned &amp; others"
4,#Trump https://t.co/3eJ3UXFvYX


In [87]:
# minimize to 10000 rows
# df = df.iloc[:10000]
#df.to_csv("output/2020_LD_1.csv", index=False) # for labelling and later fine-tuning

In [88]:
# df = pd.read_csv("output/2020_Sample_Complete.csv") # for analysis
# df = df.iloc[:300000]

In [89]:
#rename cloumn text to tweet 
df = df.rename(columns={'text': 'tweet'})

### **Duplicates, Null and Length Handling**





In [117]:
duplicates= df[df.duplicated(subset=['tweet'], keep=False)]
print("Number of duplicate tweet rows:", len(duplicates))
df = df.drop_duplicates(subset=['tweet'], keep='first').reset_index(drop=True)
print("Number of rows after removing duplicates:", len(df))

Number of duplicate tweet rows: 335877
Number of rows after removing duplicates: 1052685


In [118]:
# # for 2024: minimize to 4000 rows 
# df = df.iloc[:6000]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052685 entries, 0 to 1052684
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   tweet   1052684 non-null  object
dtypes: object(1)
memory usage: 8.0+ MB


**Handle Null Values**

In [ ]:
print("Num of null values in DataFrame:")
# pandas isnull() method to check for null values in the DataFrame
# The null values will help to identify any missing data that may need to be addressed before analysis and machine learning tasks
df.isnull().sum()
df = df.dropna(subset=['tweet']).reset_index(drop=True)
print("Number of rows after removing null tweet rows:", len(df))

Num of null values in DataFrame
Number of rows after removing null tweet rows: 1052684


**Handle Text Lenghth**

In [120]:
# remove tweets with less than 2 words to ensure meaningful analysis
# Initialize text_list from the 'tweet' column in the DataFrame
text_list = df['tweet'].tolist() 
text_list = sorted(text_list, key=lambda x: len(str(x)))

# Display top 5 shortest tweets
print("Top 5 shortest tweets:")
for tweet in text_list[:5]:
    wrap_text = textwrap.fill(str(tweet), width=80)
    print(wrap_text)
    print("-" * 50)
# drop tweets with with just two words from dataframe
df = df[df['tweet'].str.split().str.len() > 2]
# Display the shape of the DataFrame after dropping short tweets
print("Shape of the DataFrame after dropping short tweets:", df.shape) 

Top 5 shortest tweets:
0.0
--------------------------------------------------
79.0
--------------------------------------------------
690.0
--------------------------------------------------
695.0
--------------------------------------------------
688.0
--------------------------------------------------
Shape of the DataFrame after dropping short tweets: (1040118, 1)


## **Normilazation**


**Normalize whitespace (\n → space)**

In [121]:
df['tweet']= df['tweet'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [122]:
df.head()

,tweet
0,Tricky #Biden. #Elections2020 https://t.co/syPe9u7JgF
1,#HumanRights #StopArmenianAggression #Paris #StopArmenianTerrorism #Trump #Germania https://t.co/XgSKkDeiPi
2,The Conservative Movement has become a dangerous cult. Republicans are NOT going to listen to health experts??? Wow. It will be a long time getting over this pandemic. #Trump #COVIDIOTS #Cult45 #TrumpIsCompromised #VoteThemAllOut #BlueWave https://t.co/7uSF58DRta
3,"As it turns out, #Trump was right as usual. He said the problem w/tearing down statues is... where does it end? Who'll be next George Washington? Three years later, that's exactly what's happened. Lunatics want to tear down even Lincoln's statue. Washington was mentioned &amp; others"
5,#Trump is going crazy pretending people's choice to be ignored This is just insane !!! #Election2020


**Ellipsis, comma and quotations**
- (…, U+2026) into three dots (...)

In [123]:
def normalizeText(text):
    if not isinstance(text, str):
        return text
    
    # Normalize ellipsis (…) to "..."
    text = text.replace("…", "...")
    
    # Normalize curly quotes to straight quotes
    text = text.replace("“", '"').replace("”", '"')
    text = text.replace("‘", "'").replace("’", "'")
    
    # Normalize en-dash and em-dash to plain dash
    text = text.replace("–", "-").replace("—", "-")
    
    # Collapse multiple dots into three max (avoid "......")
    text = re.sub(r"\.{3,}", "...", text)
    
    return text


In [124]:
df['tweet']= df['tweet'].apply(normalizeText)
df.head()

,tweet
0,Tricky #Biden. #Elections2020 https://t.co/syPe9u7JgF
1,#HumanRights #StopArmenianAggression #Paris #StopArmenianTerrorism #Trump #Germania https://t.co/XgSKkDeiPi
2,The Conservative Movement has become a dangerous cult. Republicans are NOT going to listen to health experts??? Wow. It will be a long time getting over this pandemic. #Trump #COVIDIOTS #Cult45 #TrumpIsCompromised #VoteThemAllOut #BlueWave https://t.co/7uSF58DRta
3,"As it turns out, #Trump was right as usual. He said the problem w/tearing down statues is... where does it end? Who'll be next George Washington? Three years later, that's exactly what's happened. Lunatics want to tear down even Lincoln's statue. Washington was mentioned &amp; others"
5,#Trump is going crazy pretending people's choice to be ignored This is just insane !!! #Election2020


**Handle Non-English Tweets**


In [ ]:
DetectorFactory.seed = 0

urlPattern = re.compile(r'https?://\S+')
mentionPattern = re.compile(r'@\w+')

def pre_clean(series: pd.Series):
    cleaned = series.astype(str)
    cleaned = cleaned.str.replace(urlPattern, ' ', regex=True)
    cleaned = cleaned.str.replace(mentionPattern, ' ', regex=True)
    num_hash = cleaned.str.count('#')
    cleaned = cleaned.apply(lambda s: ''.join(' ' if ch in EMOJI_DATA else ch for ch in s).strip())
    return cleaned, num_hash

def detect_lang_safe(text):
    try:
        return detect_langs(text or "a")[0]
    except Exception:
        return None

def removeLanguage(df, col="text", verbose=True, min_prob=0.55, workers=None):
    cleaned, num_hash = pre_clean(df[col])

    mask = pd.Series(True, index=df.index)
    mask &= ~((num_hash >= 3) & (cleaned.str.split().str.len() < 4))

    subset = cleaned[mask]

    cache = {}
    def detect_with_cache(text):
        if text in cache:
            return cache[text]
        best = detect_lang_safe(text)
        cache[text] = best
        return best

    if workers and len(subset) > 0:
        def detect_batch(chunk):
            DetectorFactory.seed = 0
            return [detect_with_cache(text) for text in chunk]
        with ProcessPoolExecutor(max_workers=workers) as pool:
            results = sum(pool.map(detect_batch, np.array_split(subset.values, workers)), [])
    else:
        results = [detect_with_cache(text) for text in subset.values]

    detected = []
    keep = pd.Series(False, index=subset.index)
    for idx, best in zip(subset.index, results):
        if best is None:
            detected.append(("error", 0.0))
            continue
        lang, prob = best.lang, best.prob
        detected.append((lang, prob))
        if lang == "en" and prob >= min_prob:
            keep.at[idx] = True

    filtered = df.loc[keep.reindex(df.index, fill_value=False)].copy()

    if verbose:
        tallies = Counter(lang for lang, _ in detected)
        print("\nSummary of Language Found:")
        for lang, count in tallies.most_common():
            print(f"{lang}: {count}")
        print(f"\nRemaining rows: {len(filtered)} / {len(df)}")

    return filtered

In [126]:
#remove if not english and not hashtag only
df = removeLanguage(df, col="tweet", verbose=True)
# remove from df "tweet" column so that index that are english and not hashtag only are kept
df = df[df['tweet'].notna()].reset_index(drop=True)



Summary of Language Found:
en: 757379
es: 59292
de: 43263
fr: 36945
nl: 28951
it: 22014
tr: 13081
pt: 8978
ro: 7084
id: 5948
da: 4603
vi: 4505
ca: 3947
no: 2811
ar: 2791
af: 2638
hi: 2096
pl: 1840
fi: 1730
sv: 1700
fa: 1675
ja: 1383
ta: 1312
el: 1104
ur: 1057
lv: 968
tl: 831
et: 735
hr: 525
so: 470
te: 431
cs: 398
ml: 380
kn: 356
ru: 339
mr: 321
sw: 277
th: 276
sl: 262
gu: 259
lt: 253
bn: 225
error: 174
cy: 163
hu: 129
ko: 127
sq: 127
sk: 115
ne: 100
zh-cn: 87
pa: 57
he: 49
bg: 47
zh-tw: 41
uk: 26
mk: 11

Remaining rows: 755351 / 1040118


In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755351 entries, 0 to 755350
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   tweet   755351 non-null  object
dtypes: object(1)
memory usage: 5.8+ MB


In [129]:
df.head()

,tweet
0,Tricky #Biden. #Elections2020 https://t.co/syPe9u7JgF
1,#HumanRights #StopArmenianAggression #Paris #StopArmenianTerrorism #Trump #Germania https://t.co/XgSKkDeiPi
2,The Conservative Movement has become a dangerous cult. Republicans are NOT going to listen to health experts??? Wow. It will be a long time getting over this pandemic. #Trump #COVIDIOTS #Cult45 #TrumpIsCompromised #VoteThemAllOut #BlueWave https://t.co/7uSF58DRta
3,"As it turns out, #Trump was right as usual. He said the problem w/tearing down statues is... where does it end? Who'll be next George Washington? Three years later, that's exactly what's happened. Lunatics want to tear down even Lincoln's statue. Washington was mentioned &amp; others"
4,#Trump is going crazy pretending people's choice to be ignored This is just insane !!! #Election2020


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755351 entries, 0 to 755350
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   tweet   755351 non-null  object
dtypes: object(1)
memory usage: 5.8+ MB


### **Save to CSV**

In [131]:
# data ready for labelling and fine-tuning
#df.to_csv('2020_LD_1.csv', index=False)
# data ready for processing and analysis
df.to_csv('2020_Sample_Complete_Clean.csv', index=False)

Deepak, 2017. nltk - wordnet lemmatization and pos tagging in python - Stack Overflow. [online] 
Available at: <https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python> [Accessed 26 June 2025].
Dutta, T., 2024. Sentiment Analysis of Customer Reviews: A Guide for Beginners | by Tanushree Dutta | Medium. [online] 
Available at: <https://medium.com/@14duttatanushreeya/sentiment-analysis-of-customer-reviews-a-guide-for-beginners-4a5f2194a121> 
GeeksforGeeks, 2024a. How to Remove HTML Tags from String in Python - GeeksforGeeks. [online] Available at: <https://www.geeksforgeeks.org/python/how-to-remove-html-tags-from-string-in-python/> [Accessed 25 June 2025].
GeeksforGeeks, 2024b. Python | Lemmatization with NLTK - GeeksforGeeks. [online] Available at: <https://www.geeksforgeeks.org/python-lemmatization-with-nltk/> [Accessed 26 June 2025].
Lane, K., 2021. Tokenization with NLTK. When it comes to NLP, tokenization is a… | by Kelsey Lane | Medium. [online] Available at: <https://medium.com/%40kelsklane/tokenization-with-nltk-52cd7b88c7d> [Accessed 26 June 2025].
NLTK Project, 2024. NLTK :: Natural Language Toolkit. [online] 
Available at: <https://www.nltk.org/> [Accessed 27 June 2025].
Richardson, L., 2025. Beautiful Soup Documentation — Beautiful Soup 4.13.0 documentation. [online] 
Available at: <https://www.crummy.com/software/BeautifulSoup/bs4/doc/> [Accessed 27 June 2025].